In [1]:
import fitz
import cv2
import numpy as np
import re
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import cross_val_score
import keras_ocr
import matplotlib.pyplot as plt
import cv2
import numpy as np
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, ConfusionMatrixDisplay
import joblib

2024-05-15 00:00:02.584811: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-15 00:00:03.700810: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
class Test_Data:
    def __init__(self,pdf=None):
        self.pdf = pdf
        self.image_path = None
        self.features_array = None
        self.words_list=[]
        self.word_box_list=[]
        

   

        
    def convert_to_img(self):
        # Path to your PDF file
        pdf_path = self.pdf

        # Extract the PDF file name without extension
        pdf_name = os.path.splitext(os.path.basename(pdf_path))[0]

        # Open the PDF file
        pdf_document = fitz.open(pdf_path)

        # Iterate through each page in the PDF
        for page_number in range(len(pdf_document)):
            # Get the page
            page = pdf_document.load_page(page_number)

            # Convert the page to an image
            pix = page.get_pixmap()

            # Save the image with PDF file name added
            image_path = f'{pdf_name}_page_{page_number + 1}.png'
            pix.save(image_path)
            self.image_path = image_path

        # Close the PDF document
        pdf_document.close()




    def features_data(self):
        model = VGG16(weights='imagenet', include_top=False)

        # Initialize the OCR pipeline
        pipeline = keras_ocr.pipeline.Pipeline()
    
        # Read the image
        img_path = self.image_path
        image = keras_ocr.tools.read(img_path)

        if isinstance(image, list) and len(image) > 0:
            image = image[0]  # Assuming the first element is the image data

        # Perform OCR recognition
        prediction_groups = pipeline.recognize([image])

        # labels_list = []
        features_list = []

        # Extract font size from the LaTeX file
        

        
        for predictions in prediction_groups:
            for word, box in predictions:
                # Convert box coordinates to integers
                box = np.array(box, dtype=np.int32)
                
                box[2][1] += 3

                # Extract region of interest (ROI) from the image
                x_min, y_min = int(np.min(box[:, 0])), int(np.min(box[:, 1]))
                x_max, y_max = int(np.max(box[:, 0])), int(np.max(box[:, 1]))

                # Ensure that the bounding box coordinates are within the image boundaries
                x_min = max(0, x_min)
                y_min = max(0, y_min)
                x_max = min(image.shape[1], x_max)
                y_max = min(image.shape[0], y_max)

                roi = image[y_min:y_max, x_min:x_max]

                # Preprocess the ROI image for VGG16 model
                roi_resized = cv2.resize(roi, (224, 224))  # Resize to match VGG16 input size
                roi_resized = np.expand_dims(roi_resized, axis=0)  # Add batch dimension
                roi_preprocessed = preprocess_input(roi_resized)  # Preprocess input for VGG16

                # Extract features using VGG16 model
                features = model.predict(roi_preprocessed)

                # Store features and labels
                # features_list.append(features)
                features_list.append((word, features))
                self.word_box_list.append((word, box))


        # self.features_array = np.array(features_list)
        self.features_array = np.array([f[1] for f in features_list])  # Extract only features for array
        self.words_list = [f[0] for f in features_list]  # Extract words for list


    #for making prediction based using trained model and extracted features
    def prediction(self,model_style,model_size):
        self.convert_to_img()
        self.features_data()
        features = self.features_array
        features_flattened = features.reshape(features.shape[0], -1)
        features_reshaped = features_flattened.reshape((-1, 112, 224, 1))
        y_pred_svm = model_style.predict(features_flattened)
        predicted_font_sizes = model_size.predict(features_reshaped)
        
        words_list = self.words_list

        word_label_mapping = [(word,label,size) for word,label,size in zip(words_list, y_pred_svm,predicted_font_sizes)]
        w_b_list=self.word_box_list

        print("Number of word-box tuples:", len(w_b_list))

        return word_label_mapping,w_b_list

In [3]:
model_style= joblib.load('svm_8k.pkl')
model_size=load_model('font_size_reg.keras')

2024-05-15 00:00:05.925136: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2024-05-15 00:00:05.925166: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: aditechbuddy-IdeaPad-Gaming-3-15ACH6
2024-05-15 00:00:05.925173: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: aditechbuddy-IdeaPad-Gaming-3-15ACH6
2024-05-15 00:00:05.925302: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 535.171.4
2024-05-15 00:00:05.925321: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 535.171.4
2024-05-15 00:00:05.925326: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:309] kernel version seems to match DSO: 535.171.4


In [4]:
# pdf_path='testing_pdfs/test1.pdf'
pdf_path='image.pdf'
data=Test_Data(pdf_path)  
word_label_mapping,w_b_list = data.prediction(model_style,model_size)

# print(word_label_mapping) #label size and font style(predicted)
# print(w_b_list) #word and box size

Looking for /home/adi_techbuddy/.keras-ocr/craft_mlt_25k.h5
Looking for /home/adi_techbuddy/.keras-ocr/crnn_kurapan.h5


2024-05-15 00:00:09.427070: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 513013760 exceeds 10% of free system memory.
2024-05-15 00:00:09.590988: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 513013760 exceeds 10% of free system memory.
2024-05-15 00:00:09.879265: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 513013760 exceeds 10% of free system memory.
2024-05-15 00:00:10.286954: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 513013760 exceeds 10% of free system memory.
2024-05-15 00:00:10.551979: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 128253440 exceeds 10% of free system memory.


4/4 [==============================] - 0s 81ms/step
Number of word-box tuples: 121


In [5]:
merged_list = []

# Iterate over both lists simultaneously using zip
for (word, box), (word_label,label,size) in zip(w_b_list, word_label_mapping):
    # Ensure that the words match
    assert word == word_label, "Words do not match between the two lists"

    size_str = str(np.round(size)[0])


    # Append a tuple containing word, box, label, and features
    merged_list.append((word, box,label,size_str))  

# Print the number of elements in the merged list
print("Number of elements in the merged list:", len(merged_list))

Number of elements in the merged list: 121


In [6]:
print(merged_list)

[('administration', array([[201, 142],
       [311, 141],
       [312, 162],
       [202, 161]], dtype=int32), 'underline', '18.0'), ('branch', array([[312, 142],
       [368, 142],
       [368, 164],
       [312, 161]], dtype=int32), 'underline', '15.0'), ('11', array([[446, 204],
       [465, 204],
       [465, 224],
       [446, 221]], dtype=int32), 'normal', '10.0'), ('2010', array([[466, 204],
       [497, 204],
       [497, 223],
       [466, 220]], dtype=int32), 'normal', '15.0'), ('12014', array([[196, 205],
       [230, 205],
       [230, 224],
       [196, 221]], dtype=int32), 'normal', '15.0'), ('june', array([[415, 205],
       [445, 205],
       [445, 223],
       [415, 220]], dtype=int32), 'italic', '12.0'), ('file', array([[ 72, 206],
       [ 94, 206],
       [ 94, 225],
       [ 72, 222]], dtype=int32), 'normal', '13.0'), ('no', array([[ 96, 206],
       [115, 206],
       [115, 225],
       [ 96, 222]], dtype=int32), 'normal', '11.0'), ('admn', array([[114, 206],
    

<h2>Drawing boxes

In [7]:
#for arrow and font charaterstics
import random
# image = cv2.imread('example_page_1.png')
# image=cv2.imread('test7_page_1.png')
image=cv2.imread('image_page_1.png')

arrow_color = (0, 0, 255)  # White color for arrows
arrow_thickness = 1

dirn_r=True

# Iterate over each word in the merged dictionary
for i,(word, box, label, size) in enumerate(merged_list):
    # Extract box coordinates and predicted font
    
    box = box.astype(int)


    if(i==0):
        y1=min(box[0][1],box[1][1])
        y2=min(box[2][1],box[3][1])
        l=label


    if((min(box[0][1],box[1][1])-y1)<30):
        # for keeping font same 
        label=l
    else:
        l=label


    if(min(box[0][1],box[1][1])-y1>10):
        # to change line so changing the y coordinate to bring uniformity
        print(f"line change at word: {word}")
        y1=min(box[0][1],box[1][1])
        y2=min(box[2][1],box[3][1])

    
    
    box[0][1]=y1
    box[1][1]=y1

    box[2][1]=y2
    box[3][1]=y2


    x = box[0][0]
    y = box[0][1]
    box_width = box[1][0] - box[0][0]


    box[2][1]+=1


    # Draw bounding box rectangle on the image
    cv2.rectangle(image, (box[0][0], box[0][1]), (box[2][0], box[2][1]), (0, 255, 0), 2)

    text_position = (box[0][0], box[0][1] - 20)
    # arrow_start = (text_position[0] - random.randint(1,100), text_position[1])
    arrow_end = (box[0][0], box[0][1])


    if dirn_r:
        arrow_start = (text_position[0] + random.randint(50, 100), text_position[1])
        dirn_r=False
    else:
        arrow_start = (text_position[0] - random.randint(50, 100), text_position[1])
        dirn_r=True
#  

    
    if label=='bold':
        a='b'
        cv2.putText(image, f"{a}{word}", arrow_start,
            cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255,0,0), 1)
    
    elif label=='normal':
        a='n'
        cv2.putText(image, f"{a}{word}",arrow_start,
            cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0,255,0), 1)
    
    elif label=='italic':
        a='i'
        cv2.putText(image, f"{a}{word}", arrow_start,
            cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0,0,255), 1)
    

    elif label=='underline':
        a='u'
        cv2.putText(image, f"{a}{word}", arrow_start,
            cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0,0,0), 1)
    
    
    cv2.arrowedLine(image, arrow_start, arrow_end, arrow_color, arrow_thickness)



    
cv2.imwrite('eg4_witharr.jpg',image)

line change at word: 11
line change at word: office
line change at word: prof
line change at word: nonth
line change at word: and
line change at word: she
line change at word: nodal
line change at word: safety
line change at word: wellare
line change at word: urther
line change at word: whichever
line change at word: anthony
line change at word: dy
line change at word: to
line change at word: prof
line change at word: csssisss
line change at word: jnu


True

In [ ]:
# pip install googletrans==3.1.0a0

<h2>Translation on the image


In [ ]:
# from googletrans import Translator
# translator=Translator()
# word='hello'

# conv_word = translator.translate(word, dest="hi")
# print(conv_word.text)

In [ ]:
# from PIL import Image, ImageDraw, ImageFont

In [ ]:
# print(merged_list[0][1])
# print(merged_list[0][1][1][0])

In [ ]:
# for word, box, label, size in merged_list:
    # print(word)

<h2>Clearing background from text


In [9]:

import matplotlib.pyplot as plt
import keras_ocr
import cv2
import math
import numpy as np

#General Approach.....
#Use keras OCR to detect text, define a mask around the text, and inpaint the
#masked regions to remove the text.
#To apply the mask we need to provide the coordinates of the starting and 
#the ending points of the line, and the thickness of the line

#The start point will be the mid-point between the top-left corner and 
#the bottom-left corner of the box. 
#the end point will be the mid-point between the top-right corner and the bottom-right corner.
#The following function does exactly that.
def midpoint(x1, y1, x2, y2):
    x_mid = int((x1 + x2)/2)
    y_mid = int((y1 + y2)/2)
    return (x_mid, y_mid)

#Main function that detects text and inpaints. 
#Inputs are the image path and kreas_ocr pipeline
def inpaint_text(img_path, pipeline):
    # read the image 
    img = keras_ocr.tools.read(img_path) 
    
    # Recogize text (and corresponding regions)
    # Each list of predictions in prediction_groups is a list of
    # (word, box) tuples. 
    prediction_groups = pipeline.recognize([img])
    
    #Define the mask for inpainting
    mask = np.zeros(img.shape[:2], dtype="uint8")
    for box in prediction_groups[0]:
        x0, y0 = box[1][0]
        x1, y1 = box[1][1] 
        x2, y2 = box[1][2]
        x3, y3 = box[1][3] 
        
        x_mid0, y_mid0 = midpoint(x1, y1, x2, y2)
        x_mid1, y_mi1 = midpoint(x0, y0, x3, y3)
        
        #For the line thickness, we will calculate the length of the line between 
        #the top-left corner and the bottom-left corner.
        thickness = int(math.sqrt( (x2 - x1)**2 + (y2 - y1)**2 ))
        
        #Define the line and inpaint
        cv2.line(mask, (x_mid0, y_mid0), (x_mid1, y_mi1), 255,    
        thickness)
        inpainted_img = cv2.inpaint(img, mask, 7, cv2.INPAINT_NS)
                 
    return(inpainted_img)

# keras-ocr will automatically download pretrained
# weights for the detector and recognizer.
pipeline = keras_ocr.pipeline.Pipeline()

# img_text_removed = inpaint_text('test2_page_1.png', pipeline)
img_text_removed=inpaint_text('image_page_1.png',pipeline)

# plt.imshow(img_text_removed)


cv2.imwrite('text_removed_image.jpg', cv2.cvtColor(img_text_removed, cv2.COLOR_BGR2RGB))


Looking for /home/adi_techbuddy/.keras-ocr/craft_mlt_25k.h5
Looking for /home/adi_techbuddy/.keras-ocr/crnn_kurapan.h5
4/4 [==============================] - 10s 1s/step


True

In [11]:
total_size = 0
count = 0
for i, (word, box, label, size) in enumerate(merged_list):

    total_size += int(float(size))
    count += 1

average_size = total_size / count
print("Average Font Size:", average_size)

Average Font Size: 13.272727272727273


In [12]:
import cv2
from PIL import Image, ImageDraw, ImageFont
import numpy as np
from googletrans import Translator

# Initialize translator
translator = Translator()

# Load the background image with PIL
# bg_image = Image.open('test2/text_removed_image.jpg')
bg_image=img_text_removed

# Convert background image to numpy array
bg_np = np.array(bg_image)

# Load the image
image = cv2.imread('image_page_1.png')
height, width, channels = image.shape 
# to get the actual dimension of the image we want to modify



# Create a list to store the PIL images with text
images_with_text = []

ut=2

for i,(word, box, label, size) in enumerate(merged_list):

    # Extract box coordinates and predicted font
    box = box.astype(int)
    
    if(i==0):
        y1=min(box[0][1],box[1][1])
        y2=min(box[2][1],box[3][1])
        l=label


    if((min(box[0][1],box[1][1])-y1)<30):
        # for keeping font same 
        label=l
    else:
        l=label


    if(min(box[0][1],box[1][1])-y1>10):
        # to change line so changing the y coordinate to bring uniformity
        print(f"line change at word: {word}")
        y1=min(box[0][1],box[1][1])
        y2=min(box[2][1],box[3][1])

    
    
    box[0][1]=y1
    box[1][1]=y1

    box[2][1]=y2
    box[3][1]=y2


    x = box[0][0]
    y = box[0][1]
    box_width = box[1][0] - box[0][0]





    # print(word)
    conv_word = translator.translate(word, dest="en")
    text = conv_word.text

    # Define font size
    font_size=average_size
    # number_float = float(size)
    # font_size = int(number_float)


    # Font settings
    if label=='normal':
        font_path = 'fontstyle/notosans.ttf'
        # font_path = 'fontstyle/CenturyStd-Light.ttf'
    if label=='italic':
        font_path='fontstyle/notosans_italic.ttf'
        # font_path= 'fontstyle/CenturyStd-LightItalic.ttf'
    if label=='bold':
        font_path='fontstyle/notosans_bold.ttf'
        # font_path = 'fontstyle/CenturyStd-Bold.ttf'
    if label =='underline':
        font_path='fontstyle/notosans.ttf'
        # font_path='fontstyle/CenturyStd-Light.ttf'

    # # for compressing to fit
    # while True:
    #     font = ImageFont.truetype(font_path, font_size)
    #     text_width = draw.textlength(text, font=font)
        
    #     if text_width <= box_width:
    #         break  # Exit the loop if the text fits within the box width
        
    #     font_size -= 1  # Decrease font size by 1 and try again

    # # Truncate or split text to fit within box width
    # while len(text) > 0 and text_width > box_width:
    #     text = text[:-1]  # Remove the last character
    #     text_width  = draw.textlength(text, font=font)


    
    # Create PIL image and draw text
    if i==0:
        pil_image = Image.fromarray(bg_np)  # Use background image
    else:
        pil_image = Image.fromarray(images_with_text[-1])  # Create a copy of the previous image
    draw = ImageDraw.Draw(pil_image)



    
    


    font = ImageFont.truetype(font_path, font_size)
    text_color = (0, 0, 0)
    
    # Draw text on PIL image
    draw.text((x, y), text, fill=text_color, font=font)
    
    # Convert PIL image back to OpenCV format and store in the list
    images_with_text.append(cv2.cvtColor(np.array(pil_image), cv2.COLOR_RGB2BGR))


    if label == 'underline':
        # print(word)
        underline_start = (box[3][0], box[3][1]+2+ ut)
        underline_end = (box[2][0], box[3][1]+2+ut)
        cv2.line(images_with_text[-1], underline_start, underline_end, (0, 0, 0), 1)




   
    

cv2.imwrite('test_f2_eng.png', images_with_text[-1])


line change at word: 11
line change at word: office
line change at word: prof
line change at word: nonth
line change at word: and
line change at word: she
line change at word: nodal
line change at word: safety
line change at word: wellare
line change at word: urther
line change at word: whichever
line change at word: anthony
line change at word: dy
line change at word: to
line change at word: prof
line change at word: csssisss
line change at word: jnu


True